In [ ]:

#1#

train_file=   'C:\\Users\\Aleksa\\Documents\\Desktop\\report\\TRAIN.csv'
test_file=    'C:\\Users\\Aleksa\\Documents\\Desktop\\report\\TEST.csv'

#date_column_name=  'TRADE DATE'
columns=  ['TRADE_COMPANY','TRADE_REASON','TRADER'] # these are categorical columns
numerical_columns= [ 'PRICE','VOLUME']
coefficients= [0.01,0.02]
threshold= 3
delimiterr=","
######################################################################################################
import os
import pandas as pd 
import numpy as np
import datetime 
import time
import re
import math

pd.set_option('display.float_format', lambda x: '%.6f' % x)

pd.set_option('display.max_columns', 50) 
pd.set_option('display.max_rows', 1000) 
df_train=pd.read_csv(train_file,delimiter=delimiterr, encoding='latin_1', thousands='.',decimal=',', infer_datetime_format=True, dayfirst=True)
df_test=pd.read_csv(test_file,delimiter=delimiterr, encoding='latin_1', thousands='.',decimal=',', infer_datetime_format=True, dayfirst=True)
df_train['T']='train'
df_test['T']='test'
df = pd.concat([df_train, df_test])

for i in numerical_columns:
    df[str(str(i)+'_original')]=df[i]
    df[i]=df[i]-min(df[i]-0.0001) # scale the minimum deal rate to zero
#df[date_column_name]=pd.to_datetime(pd.Series(df[date_column_name]))#, format="%d-%m-%y")
df1=df

#2#

strings=[]
floats=[]
integers=[]
dates=[]
booleans=[]

f=df1.dtypes
for i in range (0, len(f)):
    if f[i] =='object':
        strings.append(i)
        
    elif f[i]=='int64':
        integers.append(i)
        
    elif f[i]=='datetime64[ns]':
        dates.append(i)
        
    elif f[i]=='float64':
        floats.append(i) 
    
    elif f[i]=='bool':
        booleans.append(i)
        

df1a=df1[columns]
df2=df1
df2['group'] = df1a['group'] = df1a.values.sum(axis=1)

for i,j in zip(range(0,len(numerical_columns)),numerical_columns): 
        df2[str(j)+'_bins']=0
        df2[str(j)+'_data']=0
        df2[str(j)+'_limits']=0


df3=     df2.loc[(df2['T']=='train')]
df3_test=df2.loc[(df2['T']=='test')]

#3#
for num,coeff in zip(numerical_columns, coefficients):
    for g in df3_test.group.unique(): 
        for f in df3.group.unique():
            if g==f:
                

                globals()[str('data_')+str(num)]=df3.loc[(df3['group']==f), num] 
                globals()[str('data1_')+str(num)]=globals()[str('data_')+str(num)].sort_values().unique()
                globals()[str('datatotal_')+str(num)]=[0] 
                for i in globals()[str('data1_')+str(num)]: 
                    gg=i*(1-coeff)
                    hh=i*(1+coeff)
                    globals()[str('datatotal_')+str(num)].append(gg)
                    globals()[str('datatotal_')+str(num)].append(hh)

                globals()[str('datatotal_')+str(num)].append(100000000)


                globals()[str('final_')+str(num)]=[] 
                for i in range(0,len(globals()[str('datatotal_')+str(num)])): 
                    if i == 0:
                        globals()[str('final_')+str(num)].append( globals()[str('datatotal_')+str(num)][i])
                    elif i != (len( globals()[str('datatotal_')+str(num)])-1):
                        if  globals()[str('datatotal_')+str(num)][i] <  globals()[str('datatotal_')+str(num)][i+1]:
                            if  globals()[str('datatotal_')+str(num)][i] >  globals()[str('datatotal_')+str(num)][i-1]:
                                globals()[str('final_')+str(num)].append( globals()[str('datatotal_')+str(num)][i])
                    else:
                        globals()[str('final_')+str(num)].append(globals()[str('datatotal_')+str(num)][i])


                globals()[str('bins_')+str(num)] =[]  
                for i in globals()[str('data_')+str(num)]:
                    if i==0:
                         globals()[str('bins_')+str(num)].append(0)
                    elif i!=0:    
                        for t in range(0,len( globals()[str('final_')+str(num)])):
                            if i>=globals()[str('final_')+str(num)][t]:
                                if i<globals()[str('final_')+str(num)][t+1]:
                                    globals()[str('bins_')+str(num)].append(t)



                globals()[str('limits_')+str(num)]=[]
                for i in globals()[str('data_')+str(num)]:
                    for t in range(0,len(globals()[str('final_')+str(num)])):
                        if i>=globals()[str('final_')+str(num)][t]:
                            if i<globals()[str('final_')+str(num)][t+1]:
                                globals()[str('limits_')+str(num)].append(str([format(globals()[str('final_')+str(num)][t],'.2f'),format(globals()[str('final_')+str(num)][t+1],'.2f')]))


                df3.loc[(df3['group']==f), str(num +'_bins')]=globals()[str('bins_')+str(num)]  
                df3.loc[(df3['group']==f), str(num +'_data')]=globals()[str('data_')+str(num)]
                df3.loc[(df3['group']==f), str(num +'_limits')]=globals()[str('limits_')+str(num)]
                
                
                
                
                globals()[str('data_test_')+str(num)]=df3_test.loc[(df3_test['group']==f), num] 
    
                globals()[str('bins_test_')+str(num)] =[] 
                for i in globals()[str('data_test_')+str(num)]:
                    if i==0:
                        globals()[str('bins_test_')+str(num)].append(0)
                    elif i!=0:    
                        for t in range(0,len(globals()[str('final_')+str(num)])):
                            if i>=globals()[str('final_')+str(num)][t]:
                                if i<globals()[str('final_')+str(num)][t+1]:
                                    globals()[str('bins_test_')+str(num)].append(t)

                globals()[str('limits_test_')+str(num)]=[] 
                for i in globals()[str('data_test_')+str(num)]:
                     for t in range(0,len(globals()[str('final_')+str(num)])):
                        if i>=globals()[str('final_')+str(num)][t]:
                            if i<globals()[str('final_')+str(num)][t+1]:
                                globals()[str('limits_test_')+str(num)].append(str([format(globals()[str('final_')+str(num)][t],'.2f'),format(globals()[str('final_')+str(num)][t+1],'.2f')]))

                df3_test.loc[(df3_test['group']==f), str(num +'_bins')]  = globals()[str('bins_test_')+str(num)] 
                df3_test.loc[(df3_test['group']==f), str(num +'_data')]  = globals()[str('data_test_')+str(num)]
                df3_test.loc[(df3_test['group']==f), str(num +'_limits')]= globals()[str('limits_test_')+str(num)] 
                
#4#
df3['ONE']=1
df3_test['ONE']=0
df_final = df3.append(df3_test) # join the training and test set
cols=[]
counts=[]
for num in numerical_columns:
    cols.append(str(num +'_bins'))
    counts.append(str(num +'_count'))
    
df_final['CONC'] = df_final[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df_final['CONC']=df_final['CONC'] +'_'+ df_final['group']
df_final['FREQ'] = df_final.groupby('CONC').ONE.transform(np.sum) 

df_final['group_count']=0
for j in numerical_columns: 
        df_final[str(j)+'_count']=0
        
        for i in df3_test['group'].unique(): 
            df_final.loc[(df_final['group']==i), str(str(j)+ '_count')]=df_final.loc[(df_final['group']==i)].groupby(str(str(j)+ '_bins')).ONE.transform(np.sum)
            df_final.loc[(df_final['group']==i),'group_count']=df_final.loc[(df_final['group']==i)].groupby('group').ONE.transform(np.sum)

#5#
df_test_final= df_final.loc[(df2['T']=='test')]
df_test_final['STATUS']='OK'

df_test_final.loc[df_test_final['FREQ'] <threshold, ['STATUS']] = 'NOT OK'


df_test_final.sort_values(by='FREQ', ascending=True)
print(df_test_final)



